In [1]:
# 读取已下载的数据到 DataFrame
from pathlib import Path
import os

from freqtrade.data.history import load_pair_history

def _find_project_root(start: Path | None = None) -> Path:
    start = start or Path.cwd()
    for p in (start, *start.parents):
        if (p / "config.json").is_file():
            return p
    raise FileNotFoundError("未找到项目根目录：从当前目录向上未发现 config.json")

PROJECT_ROOT = _find_project_root()
# 统一工作目录到项目根目录，避免相对路径（例如 user_data_dir="."）随 Notebook 运行目录漂移。
os.chdir(PROJECT_ROOT)

df = load_pair_history(datadir=Path("data/okx"), timeframe="1h", pair="BTC/USDT")
if df.empty:
    raise ValueError(f"未在 {Path('data/okx').resolve()} 读取到 BTC/USDT 1h 历史数据，请检查数据是否已下载/路径是否正确。")
df.tail()


,date,open,high,low,close,volume
714,2026-01-05 19:00:00+00:00,94451.1,94499.0,94133.6,94302.8,265.032684
715,2026-01-05 20:00:00+00:00,94302.8,94793.7,94034.2,94174.5,369.738545
716,2026-01-05 21:00:00+00:00,94175.4,94231.0,93801.8,94092.9,123.160190
717,2026-01-05 22:00:00+00:00,94092.9,94399.0,94008.0,94171.0,92.922083
718,2026-01-05 23:00:00+00:00,94171.1,94195.5,93858.4,93870.9,139.429309


In [2]:
# 分类模型
import numpy as np


def classification_tc(df):
    # 开盘价-收盘价，结果大于为阴线，小于为阳线
    df["Open-Close"] = df["open"] - df["close"]
    # 最高价-最低价，衡量这根k线的长度，价格波动大小
    df["High-Low"] = df["high"] - df["low"]
    # 目标变量，下一根k线是否涨
    df["target"] = np.where(df["close"].shift(-1) > df["close"], 1, -1)
    df = df.dropna()
    x = df[["Open-Close", "High-Low"]]
    y = df["target"]
    return x, y

In [3]:
# 拆分训练集与验证集
from sklearn.model_selection import train_test_split

# 使用classification_tc函数生成数据集的特征与目标
x, y = classification_tc(df)
# 将数据集拆分为训练集与验证集
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.2)
print(x_train, "\n", x_test)

     Open-Close  High-Low
596      -128.8     211.1
98        189.4     693.7
215       571.0     682.7
203       -54.7     328.5
268       323.7     403.2
..          ...       ...
290     -1248.7    1959.7
647      -169.9     240.3
517       -15.4     189.5
106        82.6     209.9
189        33.1     586.9

[143 rows x 2 columns] 
      Open-Close  High-Low
330        82.4     123.9
206      1012.3    1570.5
371      -274.4     734.1
715       128.3     759.5
205      1378.8    2040.0
..          ...       ...
224       -70.3     225.3
236      -189.4     405.5
248       -22.1     347.1
527      -352.0     631.0
359        28.3    1061.8

[576 rows x 2 columns]


In [4]:
# 训练模型
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=95)
# 使用KNN拟合训练集
knn_clf.fit(x_train, y_train)
# 输出模型在训练集中的准确率
print(knn_clf.score(x_train, y_train))
# 输出模型在验证集中的准确率
print(knn_clf.score(x_test, y_test))

0.45454545454545453
0.5260416666666666


In [5]:
# 模型持久化保存
from pathlib import Path

import joblib

model_path = Path("models/knn_btc_usdt_1h.pkl")
model_path.parent.mkdir(parents=True, exist_ok=True)
# 保存模型到项目根目录的 models/ 下，便于策略直接加载。
joblib.dump(knn_clf, model_path)

['models\\knn_btc_usdt_1h.pkl']

In [6]:
# 信号可视化
# 买卖点绘图
from pathlib import Path
import os

from freqtrade.configuration import Configuration
from freqtrade.data.dataprovider import DataProvider
from freqtrade.plot.plotting import generate_candlestick_graph
from freqtrade.resolvers import StrategyResolver

pair = "BTC/USDT"

try:
    PROJECT_ROOT
except NameError:
    def _find_project_root(start: Path | None = None) -> Path:
        start = start or Path.cwd()
        for p in (start, *start.parents):
            if (p / "config.json").is_file():
                return p
        raise FileNotFoundError("未找到项目根目录：从当前目录向上未发现 config.json")

    PROJECT_ROOT = _find_project_root()
    os.chdir(PROJECT_ROOT)

config = Configuration.from_files([str(PROJECT_ROOT / "config.json")])

config["strategy"] = "KNNStrategy"

strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)

strategy.ft_bot_start()

if "df" not in globals():
    raise ValueError("未找到 df 变量：请先运行最上面的“读取数据”单元。")
if df is None or df.empty:
    raise ValueError("df 为空，无法生成信号；请检查数据是否已成功加载（data/okx 下是否有数据）。")

df = strategy.analyze_ticker(df, {"pair": pair})

# 绘图
graph = generate_candlestick_graph(
    data=df, pair=pair
)
graph.show()

2026-01-06 17:51:28,080 - freqtrade.loggers - INFO - Enabling colorized output.

2026-01-06 17:51:28,082 - freqtrade.loggers - INFO - Logfile configured

2026-01-06 17:51:28,083 - freqtrade.loggers - INFO - Verbosity set to 0

2026-01-06 17:51:28,086 - freqtrade.configuration.configuration - INFO - Using user-data directory: . ...

2026-01-06 17:51:28,088 - freqtrade.configuration.configuration - INFO - Using data directory: data\okx ...

...

2026-01-06 17:51:28,467 - freqtrade.plot.plotting - INFO - Indicator "ema3" ignored. Reason: This indicator is not found in your strategy.

2026-01-06 17:51:28,468 - freqtrade.plot.plotting - INFO - Indicator "ema5" ignored. Reason: This indicator is not found in your strategy.

2026-01-06 17:51:28,469 - freqtrade.plot.plotting - WARNING - No trades found.

2026-01-06 17:51:28,479 - freqtrade.plot.plotting - INFO - Indicator "macd" ignored. Reason: This indicator is not found in your strategy.

2026-01-06 17:51:28,480 - freqtrade.plot.plotting - INFO - Indicator "macdsignal" ignored. Reason: This indicator is not found in your strategy.